<a href="https://colab.research.google.com/github/AndreasPB/ai-stuff/blob/master/bad_chat_gpt_clone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Following https://www.youtube.com/watch?v=kCc8FmEb1nY&t=160s

In [ ]:
# read it in to inspect it
with open('WikiQA-train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  3668541


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

how are glacier caves formed ?	A partly submerged glacier cave on Perito Moreno Glacier .	0
how are glacier caves formed ?	The ice facade is approximately 60 m high	0
how are glacier caves formed ?	Ice formations in the Titlis glacier cave	0
how are glacier caves formed ?	A glacier cave is a cave formed within the ice of a glacier .	1
how are glacier caves formed ?	Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice .	0
How are the directions of the velocity and force vectors related in a circular motion	In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path .	0
How are the directions of the velocity and force vectors related in a circular motion	It can be uniform , with constant angular rate of rotation ( and constant speed ) , or non-uniform with a changing rate of rotation .	0
How are the directions of the velocity and force vectors relat

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{|}~¢£§­®°±·ÁÉ×ØàáâãäæçèéêëíïðñóôöøùúûüĀāąćčĐēīłňōŚşšūƎưǎɔɪʌʔʻʾʿˀˈάέήίαβγεζηθικλμνξοπρςστυφχψωόύώАаклсяאבדהוזחטילםמןקרשरशी्ชมยรวศาิีเምሴḥṭạảấễệốồộờủỹἀἈἐἱἸὁὐὰὶὸὺ‐–—‘’“”†•‰′⁄€−≈≤≥┌┐★♀♂♋♙♟がなひらカタナ一劉勝圖安小李萬言龍원학
294


In [ ]:
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

sentence = "haaha lol xdd"
print("encoded:", encode(sentence))
print("decoded:", decode(encode(sentence)))

encoded: [72, 65, 65, 72, 65, 2, 76, 79, 76, 2, 88, 68, 68]
decoded: haaha lol xdd


In [ ]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([3668541]) torch.int64
tensor([72, 79, 87,  2, 65, 82, 69,  2, 71, 76, 65, 67, 73, 69, 82,  2, 67, 65,
        86, 69, 83,  2, 70, 79, 82, 77, 69, 68,  2, 32,  0, 34,  2, 80, 65, 82,
        84, 76, 89,  2, 83, 85, 66, 77, 69, 82, 71, 69, 68,  2, 71, 76, 65, 67,
        73, 69, 82,  2, 67, 65, 86, 69,  2, 79, 78,  2, 49, 69, 82, 73, 84, 79,
         2, 46, 79, 82, 69, 78, 79,  2, 40, 76, 65, 67, 73, 69, 82,  2, 15,  0,
        17,  1, 72, 79, 87,  2, 65, 82, 69,  2, 71, 76, 65, 67, 73, 69, 82,  2,
        67, 65, 86, 69, 83,  2, 70, 79, 82, 77, 69, 68,  2, 32,  0, 53, 72, 69,
         2, 73, 67, 69,  2, 70, 65, 67, 65, 68, 69,  2, 73, 83,  2, 65, 80, 80,
        82, 79, 88, 73, 77, 65, 84, 69, 76, 89,  2, 23, 17,  2, 77,  2, 72, 73,
        71, 72,  0, 17,  1, 72, 79, 87,  2, 65, 82, 69,  2, 71, 76, 65, 67, 73,
        69, 82,  2, 67, 65, 86, 69, 83,  2, 70, 79, 82, 77, 69, 68,  2, 32,  0,
        42, 67, 69,  2, 70, 79, 82, 77, 65, 84, 73, 79, 78, 83,  2, 73, 78,  2,
      